# Topic Modeling

## Análisis de los Episodios Nacionales: Primera Serie de Galdós

**Autora:** Alina Rojas

**Fecha de creación:** 2024-05-11

**Última fecha de modificación:** 2024-05-17

En este trabajo, se aplica el modelado de temas a un corpus de textos históricos utilizando BERTopic, una técnica avanzada de modelado de temas que aprovecha el aprendizaje profundo para identificar y agrupar automáticamente temas en grandes conjuntos de datos textuales. Este enfoque nos permite explorar y analizar patrones temáticos en los Episodios Nacionales con mayor precisión y eficiencia.

## 1 Preparación del entorno

### 1.1 Instalación de paquetes

In [1]:
!pip install bertopic
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

### 1.2 Importación de paquetes

In [2]:
# General
from google.colab import files
import pandas as pd
import csv

# Stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Lemmatization
import spacy

# Topic Modeling
from bertopic import BERTopic

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### 1.3 Cargado de los datos

In [3]:
# Cargar el archivo CSV
books_metadata_filtered = pd.read_csv('books_metadata_filtered.csv')

In [4]:
# Ver estructura y detalles del dataframe
books_metadata_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63617 entries, 0 to 63616
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         63617 non-null  object
 1   book         63617 non-null  object
 2   line_number  63617 non-null  int64 
 3   chapter      63617 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.9+ MB


In [5]:
# Ver parte del contenido del dataframe
books_metadata_filtered.head()

,text,book,line_number,chapter
0,Se me permitirá que antes de referir el gran s...,Trafalgar,7,1
1,"diga algunas palabras sobre mi infancia, expli...",Trafalgar,8,1
2,manera me llevaron los azares de la vida a pre...,Trafalgar,9,1
3,catástrofe de nuestra marina.,Trafalgar,10,1
4,"Al hablar de mi nacimiento, no imitaré a la ma...",Trafalgar,11,1


### 1.4 Preparación de los datos

A continuación, se va a eliminar las stopwords del campo `text` para la extracción de tópicos, junto con los carácteres especiales.

In [6]:
# Cargar la lista de stopwords en español
spanish_stopwords = set(stopwords.words("spanish"))

def remove_stopwords_from_text(text):
    """
    Elimina las stopwords de un texto dado.

    Parámetros:
    text (str): El texto del cual se eliminarán las stopwords.

    Retorna:
    str: Una cadena de texto con las stopwords eliminadas.

    Nota:
    Esta función asume que existe una lista 'spanish_stopwords' que contiene las stopwords en español.
    """
    words = text.split()  # Divide el texto en palabras.
    # Filtra las palabras, eliminando las que están en la lista de stopwords.
    filtered_words = [word for word in words if word.lower() not in spanish_stopwords]
    # Une las palabras filtradas en una sola cadena de texto y la retorna.
    return ' '.join(filtered_words)

In [7]:
# Reemplazar los signos de puntuación en la columna 'text', manteniendo tildes y diéresis, y eliminar stopwords
books_metadata_filtered['text_filtered'] = books_metadata_filtered['text'].str.replace(r'[^\w\s]', ' ', regex=True).apply(remove_stopwords_from_text)

In [8]:
# Ver parte del contenido del dataframe
books_metadata_filtered.head()

,text,book,line_number,chapter,text_filtered
0,Se me permitirá que antes de referir el gran s...,Trafalgar,7,1,permitirá referir gran suceso testigo
1,"diga algunas palabras sobre mi infancia, expli...",Trafalgar,8,1,diga palabras infancia explicando extraña
2,manera me llevaron los azares de la vida a pre...,Trafalgar,9,1,manera llevaron azares vida presenciar terrible
3,catástrofe de nuestra marina.,Trafalgar,10,1,catástrofe marina
4,"Al hablar de mi nacimiento, no imitaré a la ma...",Trafalgar,11,1,hablar nacimiento imitaré mayor parte


Después, se va a aplicar lemmatization con SpaCy:

- **Definición de Lemmatization**: La lematización es el proceso de reducir las palabras a su forma base o lema, teniendo en cuenta su uso específico en el texto, incluyendo la categoría gramatical y el contexto. A diferencia del «stemming», que simplemente corta partes de la palabra para llegar a una forma base, la lematización analiza la morfología completa de la palabra para llegar a su lema canónico. Por ejemplo, «corriendo» se lematiza a «correr», y «mejores» a «bueno».

- **Importancia de la Lemmatization**: Utilizar la lematización es crucial en muchos campos del procesamiento del lenguaje natural (NLP) porque permite:
  - **Mejorar la precisión del análisis**: Al reducir las palabras a su forma base, se facilita la tarea de analizar el texto y se mejora la precisión del análisis semántico y sintáctico.
  - **Aumentar la relevancia en las búsquedas**: En sistemas de recuperación de información, como motores de búsqueda y recomendadores de texto, la lematización ayuda a mejorar la relevancia de los resultados al tratar variantes de una palabra como la misma entidad.
  - **Facilitar la comparación y el agrupamiento de textos**: Al normalizar las palabras a sus lemas, es más fácil comparar, agrupar y realizar análisis estadísticos en grandes volúmenes de texto, lo que es esencial para tareas como la clasificación de documentos y el modelado de temas.
  - **Reducir la complejidad computacional**: Al disminuir la cantidad de formas verbales o declinaciones que el sistema necesita procesar, se reduce la complejidad y se ahorran recursos computacionales.

Incorporar la lematización en el procesamiento de texto con SpaCy no solo mejora la calidad de los datos analizados, sino que también amplía las posibilidades de extraer significado y valor del texto procesado.

In [9]:
# Cargar el modelo en español y deshabilitar componentes innecesarios
nlp = spacy.load('es_core_news_sm', disable=['ner', 'parser'])

def lemmatize_text(text):
    """
    Procesa un texto utilizando un modelo de lenguaje de SpaCy para convertir cada palabra a su forma lematizada,
    reteniendo nombres propios y palabras importantes.

    Parámetros:
    text (str): El texto que se desea lematizar. Debe ser una cadena de texto en español.

    Retorna:
    str: Un string que contiene el texto lematizado, donde cada palabra del texto original ha sido convertida a su forma base según el contexto y la morfología del español.
    """
    # Procesar el texto con el modelo
    doc = nlp(text)
    lemmatized_text = []
    for token in doc:
        # Retener nombres propios y palabras no incluidas en las stopwords
        if token.pos_ == 'PROPN' or token.is_stop:
            lemmatized_text.append(token.text)
        elif token.is_alpha:
            lemmatized_text.append(token.lemma_)
    return ' '.join(lemmatized_text)

# Aplicar la lematización a la columna 'text_filtered'
books_metadata_filtered['text_lemmatized'] = books_metadata_filtered['text_filtered'].apply(lemmatize_text)

In [10]:
# Mostrar los primeros resultados para verificar
print(books_metadata_filtered[['text_filtered', 'text_lemmatized']].head())

                                     text_filtered  \
0            permitirá referir gran suceso testigo   
1        diga palabras infancia explicando extraña   
2  manera llevaron azares vida presenciar terrible   
3                                catástrofe marina   
4            hablar nacimiento imitaré mayor parte   

                               text_lemmatized  
0         permitir referir gran suceso testigo  
1      decir palabra infancia explicar extraña  
2  manera llevar azar vido presenciar terrible  
3                            catástrofe marino  
4        hablar nacimiento imitaré mayor parte  


In [11]:
# Ver el contenido del dataframe
books_metadata_filtered.head()

,text,book,line_number,chapter,text_filtered,text_lemmatized
0,Se me permitirá que antes de referir el gran s...,Trafalgar,7,1,permitirá referir gran suceso testigo,permitir referir gran suceso testigo
1,"diga algunas palabras sobre mi infancia, expli...",Trafalgar,8,1,diga palabras infancia explicando extraña,decir palabra infancia explicar extraña
2,manera me llevaron los azares de la vida a pre...,Trafalgar,9,1,manera llevaron azares vida presenciar terrible,manera llevar azar vido presenciar terrible
3,catástrofe de nuestra marina.,Trafalgar,10,1,catástrofe marina,catástrofe marino
4,"Al hablar de mi nacimiento, no imitaré a la ma...",Trafalgar,11,1,hablar nacimiento imitaré mayor parte,hablar nacimiento imitaré mayor parte


In [12]:
# Combinar todos los textos en un solo corpus y guardar los libros asociados
corpus = books_metadata_filtered['text_filtered'].values.tolist()

## 2 Topic Modeling

### 2.1 Extraer los tópicos del texto

In [13]:
# Crear el modelo BERTopic
topic_model = BERTopic(language="spanish")

# Ajustar el modelo a las incrustaciones
topics, probs = topic_model.fit_transform(corpus)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


In [14]:
# Obtener una visión general de los temas generados
topic_info = topic_model.get_topic_info()
print(topic_info)

     Topic  Count                                        Name  \
0       -1  22812                      -1_usted_señor_bien_si   
1        0   1065        0_franceses_francés_francia_francesa   
2        1    717               1_muerte_morir_muertos_muerto   
3        2    714         2_soldados_ejército_militar_soldado   
4        3    538               3_armas_fusil_bala_artillería   
..     ...    ...                                         ...   
643    642     10  642_recompensa_recompensar_aguarda__sirven   
644    643     10    643_recuerdos_recuerdo_menudearon_amadas   
645    644     10   644_noche_sabedores_replegó_pertenecíamos   
646    645     10      645_verídico_hecho_conseguido_realidad   
647    646     10            646_ancianos_mujeres_viejos_sexo   

                                        Representation  \
0    [usted, señor, bien, si, hombre, tan, aquel, p...   
1    [franceses, francés, francia, francesa, france...   
2    [muerte, morir, muertos, muerto, cadáver

### 2.2 Exportar datos

In [15]:
# Función para obtener las palabras y sus pesos para un tópico dado, limpiando y agregando los pesos de palabras duplicadas dentro del tópico
def get_topic_words_weights(topic_model, topic_number, n_words):
    """
    Obtiene las palabras y sus pesos para un tópico dado, limpiando y agregando los pesos de las palabras duplicadas.

    Parámetros:
    topic_model (BERTopic): El modelo BERTopic entrenado.
    topic_number (int): El número del tópico del cual obtener las palabras y sus pesos.
    n_words (int): El número de palabras más representativas a obtener del tópico.

    Retorna:
    list: Una lista de palabras representativas del tópico.
    list: Una lista de pesos correspondientes a las palabras del tópico.
    """
    topic = topic_model.get_topic(topic_number)
    words = [(word.strip('_'), weight) for word, weight in topic[:n_words] if word.strip('_')]  # Limpiar y recoger palabras y pesos, eliminando palabras vacías
    return words

# Definir el número de tópicos y el número de palabras por tópico a obtener
top_n_topics = 50
n_words = 100

# Filtrar los tópicos más frecuentes (excluyendo el tópico -1 que suele ser "sin asignar")
top_topic_indices = topic_info[topic_info.Topic != -1].head(top_n_topics).Topic.values

# Almacenar los datos en una lista para su posterior procesamiento
data = []

# Recorrer los tópicos más frecuentes y obtener las palabras y pesos
for topic_index in top_topic_indices:
    words_weights = get_topic_words_weights(topic_model, topic_index, n_words)
    for word, weight in words_weights:
        data.append({
            'topic_index': topic_index,
            'word': word,
            'weight': weight
        })

In [16]:
# Convertir datos en DataFrame
df = pd.DataFrame(data)

# Agregar globalmente los pesos de las palabras duplicadas usando groupby y conservar la columna topic_index
df_grouped = df.groupby('word').agg({
    'topic_index': 'first',  # Conservar el primer índice de tópico encontrado
    'weight': 'mean',  # Calcular la media de los pesos
}).reset_index().sort_values(by='weight', ascending=False)

# Mostrar el DataFrame agrupado
df_grouped

,word,topic_index,weight
417,santorcaz,34,0.173070
23,amaranta,20,0.165123
199,gabriel,36,0.163413
204,gray,22,0.161716
211,guerra,37,0.152370
...,...,...,...
459,vecinos,6,0.004252
442,tertulias,6,0.004187
354,parís,0,0.004179
276,llevan,0,0.003463


In [17]:
# Guardar el dataframe en CSV
csv_filename = 'topic_models.csv'
df_grouped.to_csv(csv_filename, index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

# Descargar el archivo CSV
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Referencias

- Amy. (2022, octubre 21). *Topic Modeling with Deep Learning Using Python: BERTopic*. GrabNGoInfo. https://grabngoinfo.com/topic-modeling-with-deep-learning-using-python-bertopic/. Visitado en 2024-05-17.
